# meeting_mcp Colab / ngrok helper

This notebook prepares a Colab environment to run the `meeting_mcp` FastAPI backend and Streamlit UI, exposes them via ngrok, and copies local model files from Google Drive into the repo for faster local loading.

Run cells in order. Update paths and tokens where prompted.

In [2]:
# (Optional) Install Python dependencies. Uncomment to run in Colab if needed.
# NOTE: installing all requirements may take several minutes.
# Install from the repo requirements (adjust path if you cloned into /content)
# !pip install -r /content/AIDrivenMeetingSummaryProjectRiskDetection/requirements.txt --quiet
# Common useful packages (uncomment as needed):
# !pip install streamlit pyngrok --quiet
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --quiet
# !pip install bitsandbytes --quiet
# !pip install uvicorn --quiet
print('Skip installs if your environment already has the required packages.')

Skip installs if your environment already has the required packages.


In [3]:
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Mounted Google Drive at /content/drive')
except Exception:
    print('google.colab not available — ensure models are accessible locally (not in Drive).')


Mounted at /content/drive
Mounted Google Drive at /content/drive


In [ ]:
# Use credentials.json from Google Drive and set MCP_SERVICE_ACCOUNT_FILE
import os, shutil, pathlib

# Source on Drive (user-provided)
src = '/content/drive/MyDrive/Dissertation/Project/credentials.json'
# Destination inside the repo (recommended)
dst = '/content/AIDrivenMeetingSummaryProjectRiskDetectionNotify/meeting_mcp/config/credentials.json'

if os.path.exists(src):
    try:
        pathlib.Path(dst).parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(src, dst)
        os.environ['MCP_SERVICE_ACCOUNT_FILE'] = dst
        print('Copied credentials to', dst)
    except Exception as e:
        # If copy fails, fall back to using the Drive path directly
        os.environ['MCP_SERVICE_ACCOUNT_FILE'] = src
        print('Copy failed, using Drive path as MCP_SERVICE_ACCOUNT_FILE:', src)
        print('Error:', e)
else:
    # If the file is not present on Drive, still set the env var to the Drive path
    # so downstream code can report the missing file clearly.
    os.environ['MCP_SERVICE_ACCOUNT_FILE'] = src
    print('Credentials not found at', src)

print('MCP_SERVICE_ACCOUNT_FILE =', os.environ.get('MCP_SERVICE_ACCOUNT_FILE'))
print('Exists:', os.path.exists(os.environ.get('MCP_SERVICE_ACCOUNT_FILE')))


In [4]:
!rm -rf /content/AIDrivenMeetingSummaryProjectRiskDetectionNotify
%cd /content

/content


In [5]:
# Clone the repository into /content in Colab if not already present, and set REPO_ROOT
import os, subprocess, pathlib
repo_dir = '/content/AIDrivenMeetingSummaryProjectRiskDetectionNotify'
if not pathlib.Path(repo_dir).exists():
    print('Cloning repository to', repo_dir)
    try:
        subprocess.check_call(['git','clone','https://github.com/NaveenPalisetti/AIDrivenMeetingSummaryProjectRiskDetectionNotify.git', repo_dir])
    except Exception as e:
        print('git clone failed:', e)
else:
    print('Repository already exists at', repo_dir)
os.environ['REPO_ROOT'] = repo_dir
print('Set REPO_ROOT =', os.environ['REPO_ROOT'])
# Change working directory for the notebook runtime to the repo root
try:
    os.chdir(repo_dir)
    print('Changed CWD to', repo_dir)
except Exception as e:
    print('Failed to chdir to repo root:', e)

Cloning repository to /content/AIDrivenMeetingSummaryProjectRiskDetectionNotify
Set REPO_ROOT = /content/AIDrivenMeetingSummaryProjectRiskDetectionNotify
Changed CWD to /content/AIDrivenMeetingSummaryProjectRiskDetectionNotify


In [7]:
# Install minimal dependencies and optionally copy Mistral model from Drive to /content for faster loading.
# Uncomment the pip line when running in Colab (it may take several minutes).
# !pip install -r /content/AIDrivenMeetingSummaryProjectRiskDetection/requirements.txt --quiet
import shutil, os, pathlib
# Optional: copy Mistral model from Google Drive to local disk (adjust source path if needed)
src = '/content/drive/MyDrive/Dissertation/Project/models/mistral-7B-Instruct-v0.2'
dst = '/content/mistral-7B-Instruct-v0.2'
if os.path.exists(src):
    if not os.path.exists(dst):
        print('Copying Mistral model from Drive to local disk...')
        try:
            shutil.copytree(src, dst)
            print('Copied Mistral model to', dst)
        except Exception as e:
            print('Failed to copy Mistral model:', e)
    else:
        print('Local copy of Mistral model already exists at', dst)
else:
    print('Source Mistral model not found at', src, '— ensure Drive is mounted and path is correct')

Copying Mistral model from Drive to local disk...
Copied Mistral model to /content/mistral-7B-Instruct-v0.2


In [10]:
# Install requirements from the cloned repository (uncomment to run).
import os
# Use REPO_ROOT set by the clone cell; fall back to the usual path
repo = os.environ.get('REPO_ROOT', '/content/AIDrivenMeetingSummaryProjectRiskDetection')
print('Using repo:', repo)
# Recommended: ensure the repo path is correct before running installs
# Install package requirements from the cloned repo:
!pip install -r {repo}/requirements.txt
# Install the repository in editable mode so you can modify code without reinstalling
!pip install -e {repo}
# Heavy / GPU packages (uncomment if you need GPU support):
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install bitsandbytes --quiet
# Utility packages for the UI and tunneling
!pip install streamlit pyngrok uvicorn --quiet

Using repo: /content/AIDrivenMeetingSummaryProjectRiskDetectionNotify
Obtaining file:///content/AIDrivenMeetingSummaryProjectRiskDetectionNotify
ERROR: file:///content/AIDrivenMeetingSummaryProjectRiskDetectionNotify does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [8]:
# Mount Google Drive (only required in Colab) and set model paths.

# Default Drive paths used previously in this project — update if your Drive uses different paths
os.environ.setdefault('BART_MODEL_PATH', '/content/drive/MyDrive/Dissertation/Project/models/bart_finetuned_meeting_summary')
os.environ.setdefault('MISTRAL_MODEL_PATH', '/content/drive/MyDrive/Dissertation/Project/models/mistral-7B-Instruct-v0.2')
print('BART_MODEL_PATH =', os.environ.get('BART_MODEL_PATH'))
print('MISTRAL_MODEL_PATH =', os.environ.get('MISTRAL_MODEL_PATH'))

BART_MODEL_PATH = /content/drive/MyDrive/Dissertation/Project/models/bart_finetuned_meeting_summary
MISTRAL_MODEL_PATH = /content/drive/MyDrive/Dissertation/Project/models/mistral-7B-Instruct-v0.2


In [ ]:
# Install requirements from the repo if you prefer the workspace copy (uncomment to run).
# Adjust the path to your repo location if different.
# !pip install -r /content/drive/MyDrive/Dissertation/Project/requirements.txt --quiet
print('If you need dependencies installed, uncomment the pip command in this cell and run it.')

In [ ]:
# Prepare ngrok auth token and MCP_API_KEY (optional).
import os
if not os.environ.get('NGROK_AUTH_TOKEN'):
    try:
        v = input('Enter NGROK_AUTH_TOKEN (leave blank to skip): ').strip()
        if v:
            os.environ['NGROK_AUTH_TOKEN'] = v
    except Exception:
        pass
if not os.environ.get('MCP_API_KEY'):
    try:
        v = input('Enter MCP_API_KEY to protect the HTTP API (optional): ').strip()
        if v:
            os.environ['MCP_API_KEY'] = v
    except Exception:
        pass
print('NGROK_AUTH_TOKEN set:', bool(os.environ.get('NGROK_AUTH_TOKEN')) )
print('MCP_API_KEY set:', bool(os.environ.get('MCP_API_KEY')) )

In [ ]:
# Optional: disable API key check for local dev
import os
os.environ['DISABLE_API_KEY_CHECK'] = '1'
print('DISABLE_API_KEY_CHECK set to', os.environ.get('DISABLE_API_KEY_CHECK'))

In [ ]:
# Start FastAPI (uvicorn) in background and capture PID.
import subprocess, time, os, socket, sys
from pathlib import Path

# Prefer an explicit REPO_ROOT or GITHUB_WORKSPACE if set (useful after git clone)
repo_root = Path(os.getenv('REPO_ROOT') or os.getenv('GITHUB_WORKSPACE') or Path.cwd())
print('Using repo_root =', repo_root)
try:
    os.chdir(repo_root)
except Exception as e:
    print('Failed to chdir to repo_root:', e)

# Ensure the repo is on PYTHONPATH so imports resolve in the started process
os.environ['PYTHONPATH'] = str(repo_root)
# Keep local-dev bypass if requested earlier in the notebook
os.environ.setdefault('DISABLE_API_KEY_CHECK', os.environ.get('DISABLE_API_KEY_CHECK','1'))

UVICORN_CMD = [sys.executable, '-m', 'uvicorn', 'meeting_mcp.server.mcp_api:app', '--host', '127.0.0.1', '--port', '8000', '--reload']
logf = open('uvicorn_meeting_mcp.log','a')
try:
    proc = subprocess.Popen(UVICORN_CMD, stdout=logf, stderr=logf, env=os.environ.copy())
    print('uvicorn started, pid=', proc.pid)
    (Path('uvicorn_meeting_mcp.pid')).write_text(str(proc.pid))
except Exception as e:
    print('Failed to start uvicorn:', e)

# Wait for the server port to become ready (up to 30s)
def is_port_open(host, port):
    try:
        s = socket.create_connection((host, port), timeout=1)
        s.close()
        return True
    except Exception:
        return False

for i in range(30):
    if is_port_open('127.0.0.1', 8000):
        print('Server listening on 127.0.0.1:8000')
        break
    time.sleep(1)
else:
    print('Server did not start within 30s — tailing uvicorn log for diagnostics:')
    try:
        from subprocess import run
        run(['tail', '-n', '200', 'uvicorn_meeting_mcp.log'])
    except Exception as e:
        print('Failed to tail log:', e)

print('Waiting for FastAPI to warm up...')


In [ ]:
# Diagnostics: check uvicorn process and tail the uvicorn log
import os, time, subprocess
from pathlib import Path

pid_path = Path('uvicorn_meeting_mcp.pid')
log_path = Path('uvicorn_meeting_mcp.log')

if pid_path.exists():
    pid = pid_path.read_text().strip()
    print('uvicorn pid file:', pid_path, 'pid=', pid)
    try:
        # Process info (Linux/Colab)
        p = subprocess.run(['ps','-p', pid, '-o', 'pid,cmd'], capture_output=True, text=True)
        print(p.stdout)
    except Exception as e:
        print('ps check failed:', e)
else:
    print('uvicorn pid file not found:', pid_path)

if log_path.exists():
    print('\n--- last 200 lines of uvicorn_meeting_mcp.log ---')
    try:
        t = subprocess.run(['tail','-n','200', str(log_path)], capture_output=True, text=True)
        print(t.stdout)
    except Exception as e:
        print('tail failed:', e)
else:
    print('uvicorn log not found:', log_path)

# Quick check: attempt to connect to openapi.json on both 127.0.0.1 and 0.0.0.0
try:
    import requests
    for host in ('127.0.0.1','0.0.0.0'):
        url = f'http://{host}:8000/openapi.json'
        try:
            r = requests.get(url, timeout=3)
            print(f'Health check {host}:', r.status_code)
        except Exception as e:
            print(f'Health check {host} failed:', e)
except Exception:
    print('requests not installed — skip live health checks')

In [ ]:
# Start Streamlit UI in background and create ngrok tunnels for backend and UI.
from pyngrok import ngrok, conf
import os, subprocess, time
from pathlib import Path

if os.environ.get('NGROK_AUTH_TOKEN'):
    conf.get_default().auth_token = os.environ.get('NGROK_AUTH_TOKEN')

# Start Streamlit app (use the meeting_mcp Streamlit client)
ST_CMD = [sys.executable, '-m', 'streamlit', 'run', 'meeting_mcp/ui/streamlit_agent_client.py', '--server.port', '8501']
try:
    st_proc = subprocess.Popen(ST_CMD, stdout=open('streamlit_meeting_mcp.log','a'), stderr=open('streamlit_meeting_mcp.err','a'), env=os.environ.copy())
    print('Streamlit started, pid=', st_proc.pid)
    (Path('streamlit_meeting_mcp.pid')).write_text(str(st_proc.pid))
except Exception as e:
    print('Failed to start Streamlit:', e)

# Give Streamlit a moment
time.sleep(3)

# Kill existing tunnels to avoid duplicates
try:
    ngrok.kill()
except Exception:
    pass

print('Opening ngrok tunnel to backend (127.0.0.1:8000) ...')
backend_tunnel = ngrok.connect(addr='127.0.0.1:8000', bind_tls=True)
backend_url = backend_tunnel.public_url
print('Backend public URL:', backend_url)
os.environ['ORCHESTRATOR_BASE'] = backend_url
os.environ['ORCHESTRATOR_URL'] = backend_url + '/mcp/orchestrate'

time.sleep(1)
print('Opening ngrok tunnel to Streamlit UI (127.0.0.1:8501) ...')
ui_tunnel = ngrok.connect(addr='127.0.0.1:8501', bind_tls=True)
ui_url = ui_tunnel.public_url
print('Streamlit public URL:', ui_url)
print('Set ORCHESTRATOR_URL in your Streamlit client or use the UI URL above to access the app.')


In [ ]:
# Smoke test the orchestrator endpoint to ensure the backend responds.
import requests, json, os
backend = os.environ.get('ORCHESTRATOR_BASE') or 'http://127.0.0.1:8000'
test_url = backend + '/mcp/orchestrate'
payload = {'message':'ping','params':{}}
try:
    r = requests.post(test_url, json=payload, timeout=15)
    print('Status code:', r.status_code)
    try:
        print('Response:', json.dumps(r.json(), indent=2))
    except Exception:
        print('Non-JSON response:', r.text[:1000])
except Exception as e:
    print('Failed to call orchestrator endpoint:', e)

## Troubleshooting & next steps
- If endpoints fail, check `uvicorn_meeting_mcp.log` and `streamlit_meeting_mcp.log` for errors.
- To stop background services, run `ngrok.kill()` and kill PIDs recorded in `uvicorn_meeting_mcp.pid` and `streamlit_meeting_mcp.pid`.
- If model loading fails, verify `BART_MODEL_PATH` and `MISTRAL_MODEL_PATH` point to valid local folders containing model files (config.json, pytorch_model.bin or similar).